In [1]:
!pip install requests-oauthlib
!pip install arduino-iot-client

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 234.7/234.7 kB 3.9 MB/s eta 0:00:00


In [2]:
from oauthlib.oauth2 import BackendApplicationClient
from requests_oauthlib import OAuth2Session

import iot_api_client as iot
from iot_api_client.exceptions import ApiException
from iot_api_client.models import *
from iot_api_client.configuration import Configuration
from iot_api_client.api import DevicesV2Api, ThingsV2Api


from oauthlib.oauth2 import BackendApplicationClient
from requests_oauthlib import OAuth2Session

import iot_api_client as iot
from iot_api_client.configuration import Configuration
from iot_api_client.api import ThingsV2Api, PropertiesV2Api, SeriesV2Api
from iot_api_client.models import BatchQueryRawRequestMediaV1, BatchQueryRawRequestsMediaV1

import datetime
import csv
from datetime import datetime, timedelta


CLIENT_ID = "yZsfpRLMgZNkgAyoQYQFaOP1rHUmGcxu"
CLIENT_SECRET = "EdZoFTVBVaSv834RbXIpsVtvOJtLTem3z1y9wOiqfCxDRPBLfFm7Nl3T8jF2oUyG"
ORG_ID = "8a697149-a271-4a3e-9061-6575303ab06b"

In [3]:
def authorization():
  """
  Handles the OAuth2 authorization process and initializes API clients.

  Returns:
      tuple: A tuple containing the token, authenticated client, and the
             'our_thing' and 'our_device' objects if found.
  """
  global CLIENT_ID, CLIENT_SECRET, ORG_ID

  # Create an OAuth2 client for the backend application flow
  oauth_client = BackendApplicationClient(client_id=CLIENT_ID)
  # Create an OAuth2Session for making requests
  oauth = OAuth2Session(client=oauth_client)
  # Define the token URL
  token_url = "https://api2.arduino.cc/iot/v1/clients/token"
  # Fetch the token using the client ID, secret, and organization ID
  token = oauth.fetch_token(
      token_url=token_url,
      client_id=CLIENT_ID,
      client_secret=CLIENT_SECRET,
      include_client_id=True,
      audience="https://api2.arduino.cc/iot",
      headers={"X-Organization":ORG_ID}
  )

  # Configure and instance the API client with the obtained token
  client_config = Configuration(host="https://api2.arduino.cc")
  client_config.access_token = token.get("access_token")
  # Create the API client instance, including the organization header
  client = iot.ApiClient(client_config,header_name="X-Organization",header_value=ORG_ID)

  # as an example, interact with the devices API
  # Initialize API clients for interacting with devices and things
  devices_api = DevicesV2Api(client)
  things_api = ThingsV2Api(client)

  # Initialize variables to store our specific thing and device
  our_thing = None
  our_device = None
  try:
      # List all devices
      api_instance = DevicesV2Api(client)
      api_response = api_instance.devices_v2_list()
      # Find our specific device by ID
      for device in api_response:
          # print(device.name)
          # print(device.type)
          # print(device.id)
          if(device.id == 'dee34e1d-f046-4ab8-945e-4ff65d8eb668'):
            our_device = device

      # List all things
      things = things_api.things_v2_list()
      # Find our specific thing by ID
      for thing in things:
          if (thing.id == "8f10f4cd-8d06-4c43-bca2-990130a4c5ca"):
            our_thing = thing
          # print(f"Found thing: {tname}")

  except ApiException as e:
      # Handle API exceptions
      print("Exception when calling DevicesV2Api->devices_v2_list: %s\n" % e)

  # Return the token, client, and the found thing and device
  return token, client, our_thing, our_device

In [4]:
def display_telemetry_data_paginated(EXTRACT_FROM_STR, EXTRACT_TO_STR, sensor_one, sensor_two, interval_seconds=1000, output_filename="telemetry_data.csv"):
    # Initial authorization
    global CLIENT_ID, CLIENT_SECRET, ORG_ID

    # Call the authorization function to get token, client, thing, and device
    token, client, thing, our_device = authorization()

    # Initialize API clients for interacting with properties and time-series data
    props_api = PropertiesV2Api(client)
    series_api = SeriesV2Api(client)

    # Use 'a' mode to append to the file, or 'w' for a new file each run
    # For resuming, you might want to check if the file exists and read the last processed timestamp
    # For simplicity in this example, we'll just append.
    file_mode = 'a'
    # If the file doesn't exist, write the header
    try:
        with open(output_filename, 'x', newline='') as f:
             writer = csv.writer(f)
             writer.writerow(["device_name", "property", "Timestamp", "Value"])
    except FileExistsError:
        pass # File already exists, append data

    # Print the name of the thing being processed
    print(f"\n📡 Thing: {thing.name}")

    # List all properties associated with the thing, excluding deleted ones
    properties = props_api.properties_v2_list(id=thing.id, show_deleted=False)

    # Parse the start and end timestamps from strings
    extract_from = datetime.strptime(EXTRACT_FROM_STR, "%Y-%m-%dT%H:%M:%SZ")
    extract_to = datetime.strptime(EXTRACT_TO_STR, "%Y-%m-%dT%H:%M:%SZ")

    # Iterate through each property to check its type and fetch data
    for prop in properties:
        # Check if the property type is numeric (FLOAT or INT) as we only want telemetry data
        if prop.type in ["FLOAT", "INT"]:  # Only numeric telemetry

            # Initialize the current 'from' timestamp for pagination
            current_from = extract_from

            # Loop through time intervals until the 'to' timestamp is reached
            while current_from < extract_to:
                # Calculate the end timestamp for the current interval
                current_to = current_from + timedelta(seconds=interval_seconds)
                # Ensure the current 'to' timestamp does not exceed the overall 'to' timestamp
                if current_to > extract_to:
                    current_to = extract_to

                # Format the current 'from' and 'to' timestamps as strings
                current_from_str = current_from.strftime("%Y-%m-%dT%H:%M:%SZ")
                current_to_str = current_to.strftime("%Y-%m-%dT%H:%M:%SZ")

                print(f"    Fetching data from {current_from_str} to {current_to_str}")

                # Build the query for fetching raw time-series data for the current property and interval
                query = BatchQueryRawRequestMediaV1(
                    q=f"property.{prop.id}", # Query string targeting the specific property by ID
                    var_from=current_from_str, # Start timestamp for the query
                    to=current_to_str # End timestamp for the query
                )

                # Create a batch request containing the query
                request = BatchQueryRawRequestsMediaV1(
                    resp_version=1, # Specify the response version
                    requests=[query] # Include the single query in the requests list
                )

                try:
                    # Fetch time-series data using the batch query for the current interval
                    result = series_api.series_v2_batch_query_raw(request)
                    telemetry_data_rows = [] # Data for the current interval
                    # Process the responses from the batch query
                    for response in result.responses:
                        # Iterate through the timestamps and values returned for the property in this interval
                        for time, value in zip(response.times, response.values):
                            # Append the extracted data (thing name, property name, timestamp, value) to the list
                            telemetry_data_rows.append([thing.name, prop.name, time, value])

                    # Write data for the current interval to the file immediately
                    if telemetry_data_rows:
                        with open(output_filename, mode=file_mode, newline='') as file:
                            writer = csv.writer(file)
                            writer.writerows(telemetry_data_rows) # Write all rows from the list to the CSV
                        print(f"    Data for interval saved to {output_filename}")

                except ApiException as e:
                    # Handle API exceptions
                    if e.status == 401:
                        print("    Token expired. Re-authorizing...")
                        # Re-authorize and update the client
                        token, client, _, _ = authorization()
                        client_config = Configuration(host="https://api2.arduino.cc")
                        client_config.access_token = token.get("access_token")
                        client = iot.ApiClient(client_config, header_name="X-Organization", header_value=ORG_ID)
                        series_api = SeriesV2Api(client) # Update the API client instance
                        # Stay at the current interval to retry
                        continue
                    else:
                        print(f"    Error fetching data for interval: {e}")


                # Move to the next interval only on successful fetch
                current_from = current_to

    print(f"\nTelemetry data extraction finished.")

In [8]:
extract_from="2025-06-01T16:00:00Z"
extract_to="2025-06-15T17:00:00Z"
interval_in_seconds = 1000 # adjust as needed

display_telemetry_data_paginated(extract_from, extract_to, sensor_one = "Temperature", sensor_two = "CO2", interval_seconds=interval_in_seconds, output_filename="telemetry_data_v3.csv")


📡 Thing: ahmed_gabriel
    Fetching data from 2025-06-01T16:00:00Z to 2025-06-01T16:16:40Z


KeyboardInterrupt: 

# Archive

In [ ]:
'''

def display_telemetry_data(token, client, thing, EXTRACT_FROM, EXTRACT_TO, sensor_one, sensor_two, output_filename="telemetry_data.csv"):
    """
    Fetches telemetry data for a specific thing and saves it to a CSV file.

    Args:
        token: The authorization token.
        client: The authenticated API client instance.
        thing: The thing object for which to fetch data.
        EXTRACT_FROM: The start timestamp for data extraction (in ISO 8601 format).
        EXTRACT_TO: The end timestamp for data extraction (in ISO 8601 format).
        sensor_one: The name of the first sensor (not directly used in this function, could be for filtering).
        sensor_two: The name of the second sensor (not directly used in this function, could be for filtering).
        output_filename: The name of the CSV file to save the data to.
    """

    # Initialize API clients for interacting with properties and time-series data
    # things_api = ThingsV2Api(client) # This line is commented out and not used
    props_api = PropertiesV2Api(client)
    series_api = SeriesV2Api(client)

    # List to store data for CSV, initialized with the header row
    telemetry_data_rows = []

    # Add header row to the data list
    telemetry_data_rows.append(["device_name", "property", "Timestamp", "Value"])

    # Print the name of the thing being processed
    print(f"\n📡 Thing: {thing.name}")

    # List all properties associated with the thing, excluding deleted ones
    properties = props_api.properties_v2_list(id=thing.id, show_deleted=False)

    # Iterate through each property to check its type and fetch data
    for prop in properties:
        # Check if the property type is numeric (FLOAT or INT) as we only want telemetry data
        if prop.type in ["FLOAT", "INT"]:  # Only numeric telemetry
            print(f"  🔍 Property: {prop.name} ({prop.type})")

            # Build the query for fetching raw time-series data for the current property
            query = BatchQueryRawRequestMediaV1(
                q=f"property.{prop.id}",  # Query string targeting the specific property by ID
                var_from=EXTRACT_FROM,    # Start timestamp for the query
                to=EXTRACT_TO             # End timestamp for the query
            )

            # THE Guy above is fine # This is a comment from the original code, kept for context

            # Create a batch request containing the query
            request = BatchQueryRawRequestsMediaV1(
                resp_version=1,   # Specify the response version
                requests=[query]  # Include the single query in the requests list
            )

            print(request) # Print the request object for debugging/information
             # THE Guy above is fine # This is a comment from the original code, kept for context


            # Fetch time-series data using the batch query
            result = series_api.series_v2_batch_query_raw(request)
            print("AJHGAGDKJAFGAUJG") # Debug print statement
            print(dir(result)) # Debug print statement to show available attributes/methods of the result

            # Process the responses from the batch query
            for response in result.responses:
                # Iterate through the timestamps and values returned for the property
                for time, value in zip(response.times, response.values):
                    # Append the extracted data (thing name, property name, timestamp, value) to the list
                    telemetry_data_rows.append([thing.name, prop.name, time, value])

    # Write the collected telemetry data to the specified CSV file
    with open(output_filename, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerows(telemetry_data_rows) # Write all rows from the list to the CSV

    # Print a confirmation message indicating where the data was saved
    print(f"\nTelemetry data saved to {output_filename}")



display_telemetry_data(token, client, our_thing, extract_from, extract_to, sensor_one = "Temperature", sensor_two = "CO2", output_filename="telemetry_data_v2.csv")

'''
